# TensorFlow・Kerasのトレーニング♨

## DNN 編

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インポート](#インポート)
  - [共通関数](#共通関数)
- [DNNアルゴリズム・モデル](#)
  - [DNNで重回帰分析](#DNNで重回帰分析)
  - [DNNの２クラス分類器](#DNNの２クラス分類器)
  - [DNNの多クラス分類器](#DNNの多クラス分類器)
 

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python - DL  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python%20-%20DL

## [環境準備](TensorFlowAndKeras0.ipynb)

### インポート

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
print(tf.__version__)

import keras
print(keras.__version__)
# モデル定義
from keras.models import Model, Sequential, model_from_json, load_model
from keras.layers import Dense, Input, Activation, Flatten, Dropout, LSTM
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.optimizers import SGD, Adam
# その他
from keras.applications.vgg16 import VGG16
from keras.utils import to_categorical
from keras.utils import np_utils

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### 共通関数

#### [分類問題のメトリック表示関数](ScikitLearnTraining5.ipynb)

In [ ]:
def print_metrics(label, pred):
    print('accuracy: %.3f' % metrics.accuracy_score(label, pred)) # 正答率
    
    print('\nmicro') # ミクロ平均
    print('recall: %.3f' % metrics.recall_score(label, pred, average='micro')) # 再現率
    print('precision: %.3f' % metrics.precision_score(label, pred, average='micro')) # 適合率
    print('f1_score: %.3f' % metrics.f1_score(label, pred, average='micro')) # f値
    
    print('\nmacro') # マクロ平均
    print('recall: %.3f' % metrics.recall_score(label, pred, average='macro')) # 再現率
    print('precision: %.3f' % metrics.precision_score(label, pred, average='macro')) # 適合率
    print('f1_score: %.3f' % metrics.f1_score(label, pred, average='macro')) # f値

#### 学習履歴表示関数

##### 損失

In [ ]:
def plot_history_loss(hist):
    plt.plot(hist.history['loss'],label="loss for training")
    plt.plot(hist.history['val_loss'],label="loss for validation")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    plt.show()

###### 平均絶対値誤差

In [ ]:
def plot_history_mae(hist):
    plt.plot(hist.history['mae'],label="mae for training")
    plt.plot(hist.history['val_mae'],label="mae for validation")
    plt.title('model mae')
    plt.xlabel('epoch')
    plt.ylabel('mae')
    plt.legend(loc='best')
    plt.show()

##### 正解率

In [ ]:
def plot_history_acc(hist):
    plt.plot(hist.history['accuracy'],label="accuracy for training")
    plt.plot(hist.history['val_accuracy'],label="accuracy for validation")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc='best')
    plt.show()

## DNNアルゴリズム・モデル

### DNNで重回帰分析

#### データ

##### 生成
[重回帰分析](ScikitLearnTraining1.ipynb)で使用したデータを使用。

In [ ]:
proxies = { # プロキシ設定
"http":"http://<user_name>:<password>@<proxy_host>:<proxy_port>/",
"https":"https://<user_name>:<password>@<proxy_host>:<proxy_port>/"
}

url = 'https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?plugin=attach&pcmd=open&file=Boston.csv&refer=FrontPage'
res = requests.get(url) # , verify=False, proxies=proxies) # プロキシ
df = pd.read_csv(io.BytesIO(res.content), encoding='utf-8', sep=",")
df

##### 加工

In [ ]:
df=df.drop('Unnamed: 0',axis=1)

##### 理解
...

##### 準備

###### 説明変数・目的変数の選択・分割

In [ ]:
x_org = np.array(df.drop(['medv'], axis=1))
y_org = np.array(df.loc[:, ['medv']])

###### 正規化
axis=0で列単位（変数単位）。

In [ ]:
mean = x_org.mean(axis=0)
std = x_org.std(axis=0)
x = (x_org - mean) / std

###### 学習・テストデータの分割（ホールド・アウト法

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y_org, test_size = 0.3, random_state = 0)

#### モデリング

##### DNNの定義
- 入力層  
入力ベクトルの要素数だけ。
- 隠れ層が
  - 2つ
  - 64ノードづつ
  - 活性化関数はrelu
- 出力層  
回帰なので活性化関数は恒等関数 ≒ 無し

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu,
                       input_shape=(x_train.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
])

##### コンパイル
- 回帰の損失関数は誤差二乗和（mse ≒ mean_squared_error）
- [optimizer=Adam](TensorFlowAndKeras0.ipynb)を指定する。
- metricsは平均絶対誤差（mae ≒ mean_absolute_error）

In [ ]:
model.compile(loss='mse', optimizer=Adam(), metrics=['mae'])

##### 確認

In [ ]:
model.summary()

##### 実行

###### 学習

In [ ]:
batch_size = 20
n_epoch = 200
hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=n_epoch,
                 validation_data=(x_test, y_test),
                 verbose=0)

###### 推論

In [ ]:
y_pred = model.predict(x)

##### 評価

###### 実測・予測を表示

In [ ]:
plt.plot(y_org, color='blue') # 実測値
plt.plot(y_pred, color='red') # 予測値
plt.show()

###### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)
- 平均絶対誤差（MAE：Mean Absolute Error）
- 平均二乗誤差（MSE：Mean Squared Error）

In [ ]:
# sklearnで
from sklearn.metrics import mean_squared_error as mse

train_mse = mse(y_train, model.predict(x_train))
test_mse = mse(y_test, model.predict(x_test))

print('train_mse: %.3f' % train_mse)
print('test_mse: %.3f' % test_mse)
print('train_rmse: %.3f' % (train_mse ** (1/2)))
print('test_rmse : %.3f' % (test_mse ** (1/2)))

In [ ]:
# metricsで
print(model.metrics_names)

In [ ]:
score = model.evaluate(x, y_org, verbose=0)
print('mse:', score[0])
print('mae:', score[1])
print('rmae:', (score[0] ** (1/2)))

In [ ]:
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train mse:', train_score[0])
print('Train mae:', train_score[1])
print('Train rmae:', (train_score[0] ** (1/2)))

In [ ]:
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test mse:', test_score[0])
print('Test mae:', test_score[1])
print('Test rmae:', (test_score[0] ** (1/2)))

###### 学習履歴を表示

In [ ]:
plot_history_loss(hist)

In [ ]:
plot_history_mae(hist)

### DNNの２クラス分類器

#### データ

##### 生成

###### ダウンロードしてDFに読込

In [ ]:
proxies = { # プロキシ設定
"http":"http://<user_name>:<password>@<proxy_host>:<proxy_port>/",
"https":"https://<user_name>:<password>@<proxy_host>:<proxy_port>/"
}

url = 'https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?plugin=attach&pcmd=open&file=diabetes.csv&refer=FrontPage'
res = requests.get(url) # , verify=False, proxies=proxies) # プロキシ
df = pd.read_csv(io.BytesIO(res.content), encoding='utf-8', sep=",")
df

###### [説明を参照](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20DataSet#j443d0cb)

In [ ]:
df.columns

##### 加工
...

##### 理解

In [ ]:
# 基本統計量
df.describe()

In [ ]:
# 相関係数（相関行列）
df.corr()

In [ ]:
# ヒートマップに表示
plt.figure(figsize=(10, 10))
sns.heatmap(df.corr(), annot=True, square=True, fmt='.2f')
plt.show()

In [ ]:
# 散布図行列を表示
#sns.pairplot(df, hue='Outcome')

##### 準備

###### 説明変数・目的変数の選択・分割

In [ ]:
x_org = np.array(df.loc[:, ['Pregnancies','Glucose','BMI', 'Age']])
y_org = np.array(df.loc[:, ['Outcome']])

###### 標準化

In [ ]:
ss = StandardScaler()
x = ss.fit_transform(x_org)

# 確認
## 平均が ≒ 0
print("x.mean(): ", x.mean())
## 標準偏差が ≒ 1
print("x.std(): ", x.std())

###### 学習・テストデータの分割（ホールド・アウト法

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y_org, test_size = 0.3, random_state = 0)

#### モデリング

##### DNNの定義
- 入力層  
入力ベクトルの要素数だけ。
- 隠れ層が
  - 2つ
  - 64ノードづつ
  - 活性化関数はrelu
- 出力層  
２クラス分類なので
  - ノードは１つ（0-1）
  - 活性化関数はsigmoid

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu,
                       input_shape=(x_train.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

##### コンパイル
- 2値分類の損失関数は二値交差エントロピー（binary_crossentropy）
- [optimizer=Adam](TensorFlowAndKeras0.ipynb)を指定する。
- metricsは正解率（accuracy）

In [ ]:
# metricsにf1_score, precision, recallの評価関数を定義して指定することも可能
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

##### 確認

In [ ]:
model.summary()

#### 実行

##### 学習

In [ ]:
batch_size = 10
n_epoch = 200
hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=n_epoch,
                 validation_data=(x_test, y_test),
                 verbose=0)

##### 推論

In [ ]:
y_pred = model.predict(x)

#### 評価

###### 実測・予測を表示

In [ ]:
y_pred = np.array(np.round(y_pred), dtype=np.int64)
ret = (y_org == y_pred)
(len(np.where(ret==True)[0]) / ret.size)

###### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)

In [ ]:
print(model.metrics_names)

In [ ]:
score = model.evaluate(x, y_org, verbose=0)
print('loss:', score[0])
print('accuracy:', score[1])

In [ ]:
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])

In [ ]:
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

###### 混同行列

In [ ]:
cm_minst = confusion_matrix(y_org, y_pred)
print(cm_minst.dtype)
print(cm_minst)

In [ ]:
print_metrics(y_org, y_pred)

###### 学習履歴を表示
- がっつり過学習している。
- [過学習を抑止してスコアが上がるかどうか？](https://qiita.com/hiroyuki827/items/213146d551a6e2227810)

In [ ]:
plot_history_loss(hist)

In [ ]:
plot_history_acc(hist)

### DNNの多クラス分類器

#### データ

##### 生成
[クラス分類器](ScikitLearnTraining3.ipynb)で使用したデータを使用。

In [ ]:
iris = datasets.load_iris()
df_data = pd.DataFrame(iris.data, columns=iris.feature_names)
df_target = pd.DataFrame(iris.target, columns=['species'])
df = pd.concat([df_data, df_target], axis=1)

##### 加工
...

##### 理解
...

##### 準備

###### 説明変数・目的変数の選択・分割

In [ ]:
np_arr=np.array(df)

# n = 100で2値分類
# n = 150で3値分類
n = 150

# PetalLengthCm, PetalWidthCm列の選択
#x=np_arr[:n, 2:4] 
# 全列の選択
x=np_arr[:n, 0:4] 

# Species列の選択
y=np.array(np_arr[:n, 4:5],dtype=np.int64) # 要素の型をint64に変換

###### 標準化
カテゴリ・データは対象外

In [ ]:
ss = StandardScaler()
ss.fit(x)
x_std = ss.transform(x)

###### one-hotエンコーディング
Keras（≒ 深層学習）では正解ラベルはOne-Hotベクトル化が必要。

In [ ]:
yy = to_categorical(y)

###### 学習・テストデータの分割

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_std, yy, test_size=0.3, random_state=0)

#### モデリング

##### DNNの定義
- 入力層  
入力ベクトルの要素数だけ。
- 隠れ層が
  - 1つ
  - 100ノード
  - 活性化関数はrelu
- 出力層  
多クラス分類なので
  - ノードは分類の数だけ
  - 活性化関数はsoftmax

In [ ]:
model = Sequential([
    Dense(100, activation=tf.nn.relu,
          input_shape=(x_train.shape[1],)),
    Dense(3, activation=tf.nn.softmax)
])

##### コンパイル
- 多値分類の損失関数は交差エントロピー（categorical_crossentropy）
- [optimizer=Adam](TensorFlowAndKeras0.ipynb)を指定する。
- metricsは正解率（accuracy）

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.01),
    metrics=['accuracy'])

##### 確認

In [ ]:
model.summary()

#### 実行

##### 学習

In [ ]:
batch_size = 10
n_epoch = 50
hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=n_epoch,
                 validation_data=(x_test, y_test),
                 verbose=0)

##### 推論

In [ ]:
yy_pred = model.predict(x_std)

#### 評価

###### 実測・予測を表示

In [ ]:
y_pred = np.array(yy_pred.argmax(axis=-1), dtype=np.int64)
ret = (y.flatten() == y_pred)
(len(np.where(ret==True)[0]) / ret.size)

###### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)

In [ ]:
print(model.metrics_names)

In [ ]:
score = model.evaluate(x_std, yy)
print('loss:', score[0])
print('accuracy:', score[1])

In [ ]:
train_score = model.evaluate(x_train, y_train)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])

In [ ]:
test_score = model.evaluate(x_test, y_test)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

###### 混同行列
間違い易い組合せが解る。

In [ ]:
cm_minst = confusion_matrix(y, y_pred)
print(cm_minst.dtype)
print(cm_minst)

In [ ]:
print_metrics(y, y_pred)

###### 学習履歴を表示

In [ ]:
plot_history_loss(hist)

In [ ]:
plot_history_acc(hist)